Embedded Inductance Coil
===

In [1]:
from netgen.occ import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
from ngsolve import Mesh

Load the inductance coil with the connectors and the core from the external step file.

In [2]:
geo = OCCGeometry("Induktivitaet.step")
shape = geo.shape
DrawGeo(shape);

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.0, -1.782236095659691, 9…

Generating a mesh from the imported geometrie failes, due to overlapping boundaries. The connectors and the coil should share a common boundary where they are connected, but they have individual unconnected surfaces!

In [3]:
mesh = Mesh(geo.GenerateMesh(maxh=2))

NgException: Stop meshing since boundary mesh is overlapping

The parts need to be connected and the surfaces unified. This is done by the "Glue" command. It finds duplicated surfaces and repairs them.

In [4]:
newshape = Glue(shape)
newgeo = OCCGeometry(newshape)
newmesh = Mesh(newgeo.GenerateMesh(maxh=2))
Draw (newmesh);
print ("Materials:", newmesh.GetMaterials())

WebGuiWidget(value={'ngsolve_version': '6.2.2104-88-g30c997527', 'mesh_dim': 3, 'order2d': 1, 'order3d': 1, 'd…

Materials: ('Kern', 'Spule', 'Ableitung', 'Zuleitung_2')


Embedding the Induction Coil into a box of air for simulations.
=

In FEM simulations the air around the coil needs to be considered. We take the geometrie from the step file and embed it into a box of air.

In [5]:
box = Box( (-20,-25, -5), (20, 20, 25) )
air = box-newshape
air.mat("air")
embshape = Glue([air,newshape])
DrawGeo(embshape);

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.0, -2.5, 10.0], 'mesh_ra…

In [6]:
embgeo = OCCGeometry(embshape)
embmesh = Mesh(embgeo.GenerateMesh(maxh=2))
Draw (embmesh);
print (embmesh.GetMaterials())

WebGuiWidget(value={'ngsolve_version': '6.2.2104-88-g30c997527', 'mesh_dim': 3, 'order2d': 1, 'order3d': 1, 'd…

('air', 'Kern', 'Spule', 'Ableitung', 'Zuleitung_2')


In [13]:
embmesh.GetMaterials()

('air', 'Kern', 'Spule', 'Ableitung', 'Zuleitung_2')

In [14]:
embmesh.GetBoundaries()

('bc_0',
 'bc_1',
 'bc_2',
 'bc_3',
 'bc_4',
 'bc_5',
 'bc_6',
 'bc_7',
 'bc_8',
 'bc_9',
 '',
 'bc_11',
 'bc_12',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'bc_24',
 'bc_25',
 'bc_26',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'bc_38',
 'bc_39',
 'bc_40',
 'bc_41',
 '',
 'bc_43',
 '',
 '',
 '',
 '',
 '',
 '')

In [15]:
from ngsolve import HCurl


In [16]:
help(HCurl)


Help on class HCurl in module ngsolve.comp:

class HCurl(FESpace)
 |   Keyword arguments can be:
 |  order: int = 1
 |    order of finite element space
 |  complex: bool = False
 |    Set if FESpace should be complex
 |  dirichlet: regexpr
 |    Regular expression string defining the dirichlet boundary.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet = 'top|right'
 |  dirichlet_bbnd: regexpr
 |    Regular expression string defining the dirichlet bboundary,
 |    i.e. points in 2D and edges in 3D.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet_bbnd = 'top|right'
 |  dirichlet_bbbnd: regexpr
 |    Regular expression string defining the dirichlet bbboundary,
 |    i.e. points in 3D.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet_bbbnd = 'top|right'
 |  definedon: Region or regexpr
 |    FESpace is only defined on specific Region, created with mesh.Materials('regexpr')
 |    

In [17]:
fes = HCurl(embmesh)

In [18]:
fes.ndof


47414

In [19]:
import netgen.gui